# Experimentos Pareados

### Configuração

In [ ]:
# Você pode defini-las diretamente
import os
os.environ["GOOGLE_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Ou você pode usar um arquivo .env
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Tarefa

Vamos configurar uma nova tarefa! Aqui, temos um vendedor chamado Bob. Bob tem muitos negócios, então ele quer resumir o que aconteceu nesses negócios baseado em algumas transcrições de reuniões.

Bob está iterando em alguns prompts diferentes, que lhe darão transcrições legais e concisas para seus negócios.

Bob curou um dataset de suas transcrições de negócios, vamos carregá-lo. Você pode dar uma olhada no dataset também se estiver curioso! Note que este não é um dataset dourado, não há saída de referência aqui.

In [9]:
from langsmith import Client

client = Client()
dataset = client.clone_public_dataset(
  "https://smith.langchain.com/public/9078d2f1-7bef-4ba7-b795-210a17682ef9/d"
)

### Experimentos

Agora, vamos executar alguns experimentos neste dataset usando dois prompts diferentes. Vamos adicionar um avaliador que tenta pontuar quão bons são nossos resumos!

In [ ]:
from pydantic import BaseModel, Field
import sys
sys.path.append('../../')
from gemini_utils import call_gemini_chat

SUMMARIZATION_SYSTEM_PROMPT = """Você é um juiz, visando pontuar quão bem um resumo resume o conteúdo de uma transcrição"""

SUMMARIZATION_HUMAN_PROMPT = """
[A Transcrição da Reunião] {transcript}
[O Início do Resumo] {summary} [O Fim do Resumo]"""

class SummarizationScore(BaseModel):
    score: int = Field(description="""Uma pontuação de 1-5 classificando quão bom é o resumo para a transcrição fornecida, com 1 sendo um resumo ruim, e 5 sendo um ótimo resumo""")
    
def summary_score_evaluator(inputs: dict, outputs: dict) -> list:
    messages = [
        {   
            "role": "system",
            "content": SUMMARIZATION_SYSTEM_PROMPT,
        },
        {
            "role": "user",
            "content": SUMMARIZATION_HUMAN_PROMPT.format(
                transcript=inputs["transcript"],
                summary=outputs.get("output", "N/A"),
            )}
    ]
    
    response = call_gemini_chat("gemini-1.5-flash", messages, 0.0)
    
    # Extrair pontuação da resposta (simplificado)
    try:
        import re
        score_match = re.search(r'\b([1-5])\b', response)
        if score_match:
            score = int(score_match.group(1))
        else:
            score = 3  # pontuação padrão
    except:
        score = 3
    
    return {"key": "summary_score", "score": score}

Primeiro, vamos executar nosso experimento com uma boa versão do nosso prompt!

In [ ]:
# Prompt Um: Bom Prompt!
def good_summarizer(inputs: dict):
    messages = [
        {
            "role": "user",
            "content": f"Resuma concisamente esta reunião em 3 frases. Certifique-se de incluir todos os eventos importantes. Reunião: {inputs['transcript']}"
        }
    ]
    
    response = call_gemini_chat("gemini-1.5-flash", messages, 0.0)
    return response

client.evaluate(
    good_summarizer,
    data=dataset,
    evaluators=[summary_score_evaluator],
    experiment_prefix="Bom Resumidor"
)

Agora, vamos executar um experimento com uma versão pior do nosso prompt, para destacar a diferença.

In [ ]:
# Prompt Dois: Prompt Pior!
def bad_summarizer(inputs: dict):
    messages = [
        {
            "role": "user",
            "content": f"Resuma isso em uma frase. {inputs['transcript']}"
        }
    ]
    
    response = call_gemini_chat("gemini-1.5-flash", messages, 0.0)
    return response

client.evaluate(
    bad_summarizer,
    data=dataset,
    evaluators=[summary_score_evaluator],
    experiment_prefix="Resumidor Ruim"
)

### Experimento Pareado

Vamos definir uma função que comparará nossos dois experimentos. Estes são os campos aos quais as funções de avaliador pareado têm acesso:
- `inputs: dict`: Um dicionário das entradas correspondentes a um único exemplo em um dataset.
- `outputs: list[dict]`: Uma lista das saídas dict produzidas por cada experimento nas entradas dadas.
- `reference_outputs: dict`: Um dicionário das saídas de referência associadas ao exemplo, se disponível.
- `runs: list[Run]`: Uma lista dos objetos Run completos gerados pelos experimentos no exemplo dado. Use isso se precisar de acesso a passos intermediários ou metadados sobre cada execução.
- `example: Example`: O Example completo do dataset, incluindo as entradas do exemplo, saídas (se disponível) e metadata (se disponível).

Primeiro, vamos dar ao nosso LLM-as-Judge algumas instruções. No nosso caso, vamos usar diretamente LLM-as-judge para classificar qual dos resumidores é o mais útil.

Pode ser difícil classificar nossos resumidores sem uma referência de verdade fundamental, mas aqui, comparar diferentes prompts cara a cara nos dará uma noção de qual é melhor!

In [ ]:
JUDGE_SYSTEM_PROMPT = """
Por favor, aja como um juiz imparcial e avalie a qualidade dos resumos fornecidos por dois resumidores de IA para a transcrição da reunião abaixo.
Sua avaliação deve considerar fatores como a utilidade, relevância, precisão, profundidade, criatividade e nível de detalhe de seus resumos.
Comece sua avaliação comparando os dois resumos e forneça uma breve explicação.
Evite quaisquer vieses de posição e certifique-se de que a ordem na qual as respostas foram apresentadas não influencie sua decisão.
Não favoreça certos nomes dos assistentes.
Seja o mais objetivo possível. """

JUDGE_HUMAN_PROMPT = """
[A Transcrição da Reunião] {transcript}

[O Início do Resumo do Assistente A] {answer_a} [O Fim do Resumo do Assistente A]

[O Início do Resumo do Assistente B] {answer_b} [O Fim do Resumo do Assistente B]"""

Nossa função receberá um dicionário `inputs`, e uma lista de dicionários `outputs` para os diferentes experimentos que queremos comparar.

In [ ]:
from pydantic import BaseModel, Field

class Preference(BaseModel):
    preference: int = Field(description="""1 se a resposta do Assistente A for melhor baseada nos fatores acima.
2 se a resposta do Assistente B for melhor baseada nos fatores acima.
Saída 0 se for um empate.""")
    
def ranked_preference(inputs: dict, outputs: list[dict]) -> list:
    messages = [
        {   
            "role": "system",
            "content": JUDGE_SYSTEM_PROMPT,
        },
        {
            "role": "user",
            "content": JUDGE_HUMAN_PROMPT.format(
                transcript=inputs["transcript"],
                answer_a=outputs[0].get("output", "N/A"),
                answer_b=outputs[1].get("output", "N/A")
            )}
    ]
    
    response = call_gemini_chat("gemini-1.5-flash", messages, 0.0)
    
    # Extrair preferência da resposta (simplificado)
    try:
        import re
        if "assistente a" in response.lower() and "melhor" in response.lower():
            preference_score = 1
        elif "assistente b" in response.lower() and "melhor" in response.lower():
            preference_score = 2
        else:
            preference_score = 0
    except:
        preference_score = 0

    if preference_score == 1:
        scores = [1, 0]
    elif preference_score == 2:
        scores = [0, 1]
    else:
        scores = [0, 0]
    return scores

Agora vamos executar nosso experimento pareado com `evaluate()`

In [ ]:
from langsmith import evaluate

evaluate(
    ("Bom Resumidor-bafea4ec", "Resumidor Ruim-06ff299d"),  # TODO: Substitua pelos nomes/IDs dos seus experimentos
    evaluators=[ranked_preference]
)